In [12]:
pip install --upgrade pip

^C
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install geopy pygeohash


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [36]:
pip install opencage

Defaulting to user installation because normal site-packages is not writeable
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pygeohash

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark ETL Task") \
    .getOrCreate()

In [7]:
spark

In [ ]:
from opencage.geocoder import OpenCageGeocode
geocoder = OpenCageGeocode('16a2553fc34b4e64b6e4da5c5183d521')

In [12]:
import pandas as pd
from pyspark.sql.types import DoubleType

# Convert Spark DataFrame to Pandas
df_pandas = df_rest.toPandas()

# Function to fetch coordinates
def fetch_coordinates(city, country):
    if pd.notna(city) and pd.notna(country):
        try:
            results = geocoder.geocode(f"{city}, {country}", no_annotations=1)
            if results and len(results) > 0:
                return results[0]['geometry']['lat'], results[0]['geometry']['lng']
        except Exception as e:
            print(f"Error fetching coordinates for {city}, {country}: {e}")
    return None, None

# Fill missing values
for index, row in df_pandas.iterrows():
    if pd.isna(row["lat"]) or pd.isna(row["lng"]):
        lat, lng = fetch_coordinates(row["city"], row["country"])
        df_pandas.at[index, "lat"] = lat
        df_pandas.at[index, "lng"] = lng
        time.sleep(1)  # Prevent hitting API limits

# Convert NaN to None (for Spark compatibility)
df_pandas = df_pandas.where(pd.notna(df_pandas), None)

print(df_pandas)

# Convert back to Spark DataFrame with correct types
df_res_filled = spark.createDataFrame(df_pandas)

# Ensure lat/lng are DoubleType
df_res_filled = df_res_filled.withColumn("lat", col("lat").cast(DoubleType()))
df_res_filled = df_res_filled.withColumn("lng", col("lng").cast(DoubleType()))

# Show output
df_res_filled.show()

Error fetching coordinates for Dillon, US: name 'geocoder' is not defined
                id  franchise_id      franchise_name  restaurant_franchise_id  \
0     197568495625            10    The Golden Spoon                    24784   
1      17179869242            59         Azalea Cafe                    10902   
2     214748364826            27     The Corner Cafe                    92040   
3     154618822706            51        The Pizzeria                    41484   
4     163208757312            65       Chef's Corner                    96638   
...            ...           ...                 ...                      ...   
1992   94489280524            13       The Firehouse                    95399   
1993  240518168584             9     The Grill House                    93164   
1994   60129542150             7           Cafe Roma                    71555   
1995  257698037794            35     Basilica Bistro                    26468   
1996   68719476749            14  T

Py4JJavaError: An error occurred while calling o92.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 1 times, most recent failure: Lost task 0.0 in stage 10.0 (TID 296) (192.168.0.18 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:789)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scal

In [ ]:
results = geocoder.geocode('Decatur, US', no_annotations='1')
results

[{'bounds': {'northeast': {'lat': 39.9249642, 'lng': -88.8032028},
   'southwest': {'lat': 39.7867965, 'lng': -89.0323691}},
  'components': {'ISO_3166-1_alpha-2': 'US',
   'ISO_3166-1_alpha-3': 'USA',
   'ISO_3166-2': ['US-IL'],
   '_category': 'place',
   '_normalized_city': 'Decatur',
   '_type': 'city',
   'city': 'Decatur',
   'continent': 'North America',
   'country': 'United States',
   'country_code': 'us',
   'county': 'Macon County',
   'state': 'Illinois',
   'state_code': 'IL'},
  'confidence': 5,
  'formatted': 'Decatur, Illinois, United States of America',
  'geometry': {'lat': 39.8454163, 'lng': -88.9524151}},
 {'bounds': {'northeast': {'lat': 31.0795374, 'lng': -84.375778},
   'southwest': {'lat': 30.6901003, 'lng': -84.8713856}},
  'components': {'ISO_3166-1_alpha-2': 'US',
   'ISO_3166-1_alpha-3': 'USA',
   'ISO_3166-2': ['US-GA'],
   '_category': 'place',
   '_type': 'county',
   'continent': 'North America',
   'country': 'United States',
   'country_code': 'us',
 

In [8]:
import os
from pyspark.sql.types import StructType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, udf
from pyspark.sql.types import DoubleType
from opencage.geocoder import OpenCageGeocode
import time

def read_data(spark, path, file_type):
    """
    Reads all files of the specified type from all subdirectories within the provided path 
    and concatenates them into a single DataFrame. It recursively searches every subdirectory,
    no matter the depth, for files ending in '.snappy.parquet' if 'parquet' is specified as file_type.
    
    Args:
        spark (SparkSession): The Spark session instance.
        path (str): The root directory path that contains the subdirectories with the files.
        file_type (str): The type of files to read ('csv' or 'parquet').
        
    Returns:
        DataFrame: A DataFrame containing all data from the files found with the specified extension.
        
    Raises:
        ValueError: If an unsupported file type is specified or no files are found.
    """
    if file_type not in ['csv', 'parquet']:
        raise ValueError("Unsupported file type. Please use 'csv' or 'parquet'.")

    all_files = []
    # Walk through directory
    for subdir, dirs, files in os.walk(path):
        for file in files:
            # Check for file extension
            if file_type == 'csv' and file.endswith(".csv"):
                all_files.append(os.path.join(subdir, file))
            elif file_type == 'parquet' and file.endswith(".snappy.parquet"):
                all_files.append(os.path.join(subdir, file))

    # Read all files into a DataFrame
    if all_files:
        if file_type == 'csv':
            df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load(all_files)
        elif file_type == 'parquet':
            df = spark.read.parquet(all_files)
    else:
        # Provide a schema or handle the empty dataset case differently
        empty_schema = StructType([])  # Define an empty schema
        df = spark.createDataFrame([], schema=empty_schema)  # Create an empty DataFrame with a defined schema

    return df

In [9]:
# Read and concatenate data
df_rest = read_data(spark, 'data/restaurant/', 'csv')
df_rest = df_rest.withColumn("lat", col("lat").cast("double"))
df_rest = df_rest.withColumn("lng", col("lng").cast("double"))

# Show the first few rows of the DataFrame
df_rest.show()

+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|          id|franchise_id|      franchise_name|restaurant_franchise_id|country|          city|   lat|    lng|
+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|197568495625|          10|    The Golden Spoon|                  24784|     US|       Decatur|34.578|-87.021|
| 17179869242|          59|         Azalea Cafe|                  10902|     FR|         Paris|48.861|  2.368|
|214748364826|          27|     The Corner Cafe|                  92040|     US|    Rapid City| 44.08|-103.25|
|154618822706|          51|        The Pizzeria|                  41484|     AT|        Vienna|48.213| 16.413|
|163208757312|          65|       Chef's Corner|                  96638|     GB|        London|51.495| -0.191|
| 68719476763|          28|    The Spicy Pickle|                  77517|     US|      Grayling|44.657|-84.744|
|

In [10]:
import os

def list_files(startpath):
    lists = []
    for root, dirs, files in os.walk(startpath):
        for file in files:
            if file.endswith('.snappy.parquet'):
                file_path = os.path.normpath(os.path.join(root, file))  # Normalize path for OS
                lists.append(file_path)
    return lists

# List all parquet files recursively
lists = list_files('data/geo/')

# Check if we found files
if len(lists) == 0:
    print("No parquet files found!")
else:
    print(f"Found {len(lists)} parquet files. Loading them into Spark DataFrame...")

# Read all Parquet files into a single DataFrame
df_geo = spark.read.parquet(*lists)  # Unpacking list

# Show the schema and a sample of data
df_geo.printSchema()
df_geo.show(5)


Found 276 parquet files. Loading them into Spark DataFrame...
root
 |-- lng: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- avg_tmpr_f: double (nullable = true)
 |-- avg_tmpr_c: double (nullable = true)
 |-- wthr_date: string (nullable = true)

+--------+-------+----------+----------+----------+
|     lng|    lat|avg_tmpr_f|avg_tmpr_c| wthr_date|
+--------+-------+----------+----------+----------+
| -111.09|18.6251|      80.7|      27.1|2017-08-29|
|-111.042|18.6305|      80.7|      27.1|2017-08-29|
|-110.995|18.6358|      80.7|      27.1|2017-08-29|
|-110.947|18.6412|      80.9|      27.2|2017-08-29|
|  -110.9|18.6465|      80.9|      27.2|2017-08-29|
+--------+-------+----------+----------+----------+
only showing top 5 rows



In [11]:
df_rest.describe().show()

+-------+--------------------+------------------+--------------------+-----------------------+-------+------------+-----------------+------------------+
|summary|                  id|      franchise_id|      franchise_name|restaurant_franchise_id|country|        city|              lat|               lng|
+-------+--------------------+------------------+--------------------+-----------------------+-------+------------+-----------------+------------------+
|  count|                1997|              1997|                1997|                   1997|   1997|        1997|             1996|              1996|
|   mean|1.357657013519784...| 37.35703555332999|                NULL|       46911.0375563345|   NULL|        NULL|43.99021893787576|-34.52430861723447|
| stddev|7.949933220412822E10|21.855881953771444|                NULL|     31662.264749338552|   NULL|        NULL|7.142288389673525| 49.89812670573716|
|    min|                   0|                 1|        A Tasty Bite|            

In [13]:
from pyspark.sql import functions as F
import pygeohash as pgh

# Define UDF to generate geohash (4 characters long)
geohash_udf = F.udf(lambda lat, lng: pgh.encode(lat, lng, precision=8) if lat is not None and lng is not None else None)

# Ensure lat/lng are properly cast
df_res = df_rest.withColumn("lat", F.col("lat").cast("double"))
df_res = df_res.withColumn("lng", F.col("lng").cast("double"))
df_weather = df_geo.withColumn("lat", F.col("lat").cast("double"))
df_weather = df_weather.withColumn("lng", F.col("lng").cast("double"))

# Remove NULL lat/lng to avoid geohash errors
df_res = df_res.filter((F.col("lat").isNotNull()) & (F.col("lng").isNotNull()))
df_weather = df_weather.filter((F.col("lat").isNotNull()) & (F.col("lng").isNotNull()))

# Generate geohash column
df_res = df_res.withColumn("geohash", geohash_udf(F.col("lat"), F.col("lng")))
df_weather = df_weather.withColumn("geohash", geohash_udf(F.col("lat"), F.col("lng")))

# Perform LEFT JOIN on geohash
df_joined = df_res.join(df_weather, on="geohash", how="left")

# Drop duplicate lat/lng from weather data to prevent data multiplication
df_joined = df_joined.drop(df_weather.lat).drop(df_weather.lng)

# Show final result
df_joined.show()

Py4JJavaError: An error occurred while calling o154.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 297) (192.168.0.18 executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:612)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:594)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:99)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(In

In [10]:
df_res.describe().show()

+-------+--------------------+------------------+--------------------+-----------------------+-------+------------+------------------+-------------------+
|summary|                  id|      franchise_id|      franchise_name|restaurant_franchise_id|country|        city|               lat|                lng|
+-------+--------------------+------------------+--------------------+-----------------------+-------+------------+------------------+-------------------+
|  count|                1597|              1597|                1597|                   1597|   1597|        1597|              1597|               1597|
|   mean|1.360350881805635...| 37.37319974953037|                NULL|      46782.62930494678|   NULL|        NULL|43.950726361928616|-34.769134001252354|
| stddev|7.922573496885939E10|21.944072979751308|                NULL|      31608.25415208741|   NULL|        NULL| 7.283909446371843|  49.94639777654601|
|    min|                   2|                 1|        A Tasty Bite|

In [59]:
spark.sparkContext.setLogLevel("DEBUG")


In [61]:
df_joined.printSchema()

root
 |-- geohash: string (nullable = true)
 |-- id: long (nullable = true)
 |-- franchise_id: integer (nullable = true)
 |-- franchise_name: string (nullable = true)
 |-- restaurant_franchise_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- lat: double (nullable = true)
 |-- avg_tmpr_f: double (nullable = true)
 |-- avg_tmpr_c: double (nullable = true)
 |-- wthr_date: string (nullable = true)

